# Tutorial for 3DVar and 4DVar calculation

### import `deepda` package and example function `forwardModel_r`

In [1]:
import torch
import deepda
from examples.forwardModel import forwardModel_r
from math import ceil

device = "cpu"

### 3DVar Example
#### preparing required parameters

In [2]:
def H(x: torch.Tensor):
    return x

In [3]:
B = torch.eye(3, device=device)
R = torch.eye(3, device=device)
y = torch.tensor([10., 20., 30.], device=device)
xb = torch.zeros_like(y, device=device)

#### chain call supported

In [4]:
# apply_3DVar(H, B, R, xb, y, learning_rate=2, max_iterations=300)
results_3dvar = deepda.CaseBuilder().set_background_covariance_matrix(
    B
).set_observation_covariance_matrix(R).set_observations(
    y
).set_background_state(
    xb
).set_algorithm(
    deepda.Algorithms.Var3D
).set_device(
    deepda.Device.CPU
).set_observation_model(
    H
).set_learning_rate(
    2
).set_max_iterations(
    300
).execute()

Iterations: 0, J: 1400.0, Norm of J gradient: 74.83314514160156
Iterations: 1, J: 1184.0, Norm of J gradient: 62.22539520263672
Iterations: 2, J: 1017.4765014648438, Norm of J gradient: 50.39654541015625
Iterations: 3, J: 898.08154296875, Norm of J gradient: 39.807682037353516
Iterations: 4, J: 818.659912109375, Norm of J gradient: 30.810375213623047
Iterations: 5, J: 768.9251708984375, Norm of J gradient: 23.481937408447266
Iterations: 6, J: 740.5262451171875, Norm of J gradient: 18.005840301513672
Iterations: 7, J: 728.07568359375, Norm of J gradient: 14.986849784851074
Iterations: 8, J: 727.2283935546875, Norm of J gradient: 14.758960723876953
Iterations: 9, J: 733.8892211914062, Norm of J gradient: 16.46552276611328
Iterations: 10, J: 744.373046875, Norm of J gradient: 18.841018676757812
Iterations: 11, J: 755.5263061523438, Norm of J gradient: 21.076297760009766
Iterations: 12, J: 764.700927734375, Norm of J gradient: 22.75099754333496
Iterations: 13, J: 769.8867797851562, Norm of

### 4DVar Example

In [5]:
def forwardModel_wrap(x0, time, rayleigh, prandtl, b):
    return forwardModel_r(x0.ravel(), time, rayleigh, prandtl, b).T.unsqueeze_(1)

In [6]:
# We define the control parameters here
rayleigh = 35
prandtl = 10.
b = 8./3.
# rayleigh = 0.
# prandtl = 0.
# b = 0.
# initial condition for the true reference trajectory
x0 = torch.tensor([0., 1., 2.], device=device)

# integration time parameter
dt = 1.e-3      # This is time step size
T = 1.         # Total integration time, can be as short as 10 to speed things up
n_steps = ceil(T / dt)
time = torch.linspace(0., T, n_steps + 1, device=device)  # array of discrete times

# numerical integration given initial conditions and control parameters
xt = forwardModel_wrap(x0, time, rayleigh, prandtl, b)

In [7]:
sigobs = 2.  # standard deviation of the observation noise
# How often do we observe the true state?
dtobs = 0.5  # time between observations
gap = int(dtobs / dt)  # number of time steps between each observation
time_obs = torch.cat([torch.tensor([0]), time[gap::gap]])
# Generate vector of observations
R = 1e-8 * torch.diag(torch.tile(torch.tensor(sigobs**2, device=device), (x0.size(0),)))
sqrt_s = torch.sqrt(R)
# y = Hxt
y = H(xt[gap::gap, :])
# compute observation error
# noise = 0.125 * torch.randn(size=y.shape, device=device) @ sqrt_s
# y = Hxt + epsilon
# y = y + noise
y = torch.cat([x0.reshape((1, 1, -1)), y])

#### example of setting all parameters as once

In [8]:
xb = torch.zeros((3,), device=device)
# apply_4DVar(time_obs, gap, forwardModel_wrap, H, B, R, xb, y, learning_rate=7.5e-3, max_iterations=1000, args=(rayleigh, prandtl, b))
params_dict = {
    "algorithm": deepda.Algorithms.Var4D,
    "observation_model": H,
    "background_covariance_matrix": B,
    "observation_covariance_matrix": R,
    "background_state": xb,
    "observations": y,
    "forward_model": forwardModel_wrap,
    "output_sequence_length": gap + 1,
    "observation_time_steps": time_obs,
    "gaps": [gap] * (len(time_obs) - 1),
    "learning_rate": 7.5e-3,
    "args": (rayleigh, prandtl, b),
}
results_4dvar = deepda.CaseBuilder().set_parameters(params_dict).execute()

Iterations: 0, Jb: 125000000.0, Jo: 79820849152.0, J: 79945850880.0, Norm of J gradient: 626040539971584.0


Iterations: 1, Jb: 124629216.0, Jo: 62262099968.0, J: 62386728960.0, Norm of J gradient: 1012593197056.0
Iterations: 2, Jb: 124613952.0, Jo: 56024006656.0, J: 56148619264.0, Norm of J gradient: 790117351424.0
Iterations: 3, Jb: 124946080.0, Jo: 52085522432.0, J: 52210466816.0, Norm of J gradient: 668724690944.0
Iterations: 4, Jb: 125463656.0, Jo: 49304760320.0, J: 49430224896.0, Norm of J gradient: 583518781440.0
Iterations: 5, Jb: 126080072.0, Jo: 47232217088.0, J: 47358296064.0, Norm of J gradient: 517317885952.0
Iterations: 6, Jb: 126754192.0, Jo: 45636198400.0, J: 45762953216.0, Norm of J gradient: 463468036096.0
Iterations: 7, Jb: 127464568.0, Jo: 44379217920.0, J: 44506681344.0, Norm of J gradient: 418592718848.0
Iterations: 8, Jb: 128198752.0, Jo: 43372273664.0, J: 43500474368.0, Norm of J gradient: 380636200960.0
Iterations: 9, Jb: 128948824.0, Jo: 42554601472.0, J: 42683551744.0, Norm of J gradient: 348207218688.0
Iterations: 10, Jb: 129709264.0, Jo: 41883267072.0, J: 42012975